In [1]:
from fastcore.basics import *
from fastcore.parallel import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score, accuracy_score
from functools import partial
from os import cpu_count
import pandas as pd
import numpy as np

In [2]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [3]:
import h2o
from h2o.estimators import H2ORandomForestEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.20.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.20.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpxnoc79b1
  JVM stdout: /tmp/tmpxnoc79b1/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpxnoc79b1/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.3
H2O_cluster_version_age:,"1 year, 10 months and 27 days !!!"
H2O_cluster_name:,H2O_from_python_unknownUser_12n90x
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.840 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [4]:
train1=h2o.import_file("/kaggle/input/cicddos2019only11catlabelbalanced-train/CICDDoS2019Only12CatLabelBalanced_train.csv")
#test1= h2o.import_file("/kaggle/input/cicddos2019reducedcatlabelbalanced-train/CICDDoS2019CatLabel_test7Atk.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
predictors=['Bwd Packets/s', 'Fwd Act Data Packets', 'Down/Up Ratio',
       'Bwd IAT Total', 'Bwd IAT Max', 'Bwd IAT Mean',
       'Total Fwd Packets', 'Subflow Fwd Packets', 'Fwd IAT Total',
       'Bwd Packet Length Max', 'Bwd Packet Length Mean',
       'Avg Bwd Segment Size', 'Subflow Bwd Bytes',
       'Bwd Packets Length Total', 'Packet Length Min',
       'Fwd Packet Length Min', 'Bwd IAT Min', 'Flow IAT Min',
       'Fwd IAT Min', 'Init Bwd Win Bytes', 'Packet Length Variance',
       'Packet Length Std', 'Bwd Packet Length Min', 'URG Flag Count',
       'Fwd Packet Length Max', 'Subflow Fwd Bytes',
       'Fwd Packets Length Total', 'Packet Length Mean',
       'Packet Length Max', 'Init Fwd Win Bytes', 'Fwd IAT Mean',
       'Fwd IAT Std', 'Fwd IAT Max', 'Flow IAT Std', 'Flow Packets/s',
       'Flow IAT Max', 'Fwd Header Length', 'Flow Duration',
       'CWE Flag Count', 'Bwd Packet Length Std', 'Bwd Header Length',
       'Fwd Seg Size Min', 'RST Flag Count', 'Fwd PSH Flags',
       'Active Min', 'Fwd Packet Length Std', 'Protocol',
       'SYN Flag Count']

In [6]:
response='Label'

In [7]:
train1['Label']=train1['Label'].asfactor()
#test1['Label']=test1['Label'].asfactor()

In [8]:
train, valid = train1.split_frame(ratios=[.8], seed=1234)

In [24]:
drf = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=10, seed=1234,auc_type="WEIGHTED_OVO")
drf.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid)
# print('without binomial_double_trees:', drf.auc(valid=True))

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1717563328635_10


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,650.0,1085999.0,3.0,20.0,16.876923,6.0,497.0,127.58




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.23687412938433955
RMSE: 0.48669716393702106
LogLoss: 0.6282048008164726
Mean Per-Class Error: 0.24847888222106984
AUC: 0.9720201588619469
AUCPR: 0.9721447542266858

Multinomial AUC values: 


,type,first_class_domain,second_class_domain,auc
0,Benign vs Rest,Benign,None,0.999996
1,DNS vs Rest,DNS,None,0.932696
2,LDAP vs Rest,LDAP,None,0.919501
3,MSSQL vs Rest,MSSQL,None,0.910612
4,NTP vs Rest,NTP,None,0.999724
5,NetBIOS vs Rest,NetBIOS,None,0.989208
6,Portmap vs Rest,Portmap,None,0.999040
7,SNMP vs Rest,SNMP,None,0.965882
8,SSDP vs Rest,SSDP,None,0.996102
9,Syn vs Rest,Syn,None,0.999937



See the whole table with table.as_data_frame()

Multinomial auc_pr values: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_classification_error,validation_auc,validation_pr_auc
0,,2024-06-05 08:28:40,10 min 36.766 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2024-06-05 08:28:41,10 min 37.775 sec,1.0,0.494447,0.803975,0.312015,0.960268,0.959124,0.491462,0.768649,0.309231,0.961351,0.960197
2,,2024-06-05 08:28:43,10 min 38.852 sec,2.0,0.487269,0.743085,0.300006,0.962375,0.961496,0.482604,0.645959,0.286269,0.966074,0.965993
3,,2024-06-05 08:28:44,10 min 40.136 sec,3.0,0.475827,0.704344,0.276138,0.966256,0.965677,0.470430,0.610420,0.243346,0.970833,0.970802
4,,2024-06-05 08:28:50,10 min 46.337 sec,9.0,0.481110,0.649406,0.267117,0.968823,0.968628,0.475911,0.612559,0.234154,0.972982,0.973167
5,,2024-06-05 08:29:14,11 min 10.147 sec,32.0,0.488786,0.634568,0.254337,0.971076,0.971112,0.486049,0.629669,0.244615,0.973150,0.973451
6,,2024-06-05 08:29:35,11 min 30.937 sec,50.0,0.486697,0.628205,0.248731,0.972020,0.972145,0.484042,0.623114,0.243654,0.973692,0.974018



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Fwd Packet Length Max,228125.250000,1.000000,0.104834
1,Packet Length Min,198099.390625,0.868380,0.091036
2,Packet Length Mean,187155.750000,0.820408,0.086007
3,Subflow Fwd Bytes,167933.687500,0.736147,0.077173
4,Fwd Packet Length Min,159156.437500,0.697671,0.073140
5,Packet Length Max,150981.437500,0.661836,0.069383
6,Fwd Packets Length Total,115555.054688,0.506542,0.053103
7,Init Fwd Win Bytes,80992.945312,0.355037,0.037220
8,Flow Packets/s,76027.382812,0.333270,0.034938
9,Fwd Act Data Packets,71926.156250,0.315292,0.033053



See the whole table with table.as_data_frame()


In [29]:
drf.model_performance(train).auc()

0.9754458489006002

In [25]:
drf.model_performance(valid).auc()

0.97369171580935

In [27]:
drfdf=drf.cross_validation_metrics_summary().as_data_frame()
drfdf

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
0,accuracy,0.752229,0.005807,0.753057,0.755517,0.756599,0.754381,0.749277,0.750482,0.763223,0.744691,0.751699,0.743369
1,auc,0.973174,0.000813,0.974201,0.973046,0.973410,0.973586,0.972583,0.972556,0.974282,0.971714,0.973709,0.972652
2,err,0.247771,0.005807,0.246943,0.244483,0.243401,0.245619,0.250723,0.249518,0.236777,0.255309,0.248301,0.256631
3,err_count,2576.700000,57.211210,2565.000000,2548.000000,2545.000000,2551.000000,2600.000000,2589.000000,2480.000000,2681.000000,2557.000000,2651.000000
4,logloss,0.623953,0.005541,0.621560,0.625824,0.616352,0.626806,0.631214,0.619292,0.616806,0.627596,0.622388,0.631694
5,max_per_class_error,0.919135,0.035179,0.921569,0.951613,0.952880,0.940667,0.945433,0.860409,0.858779,0.935723,0.924757,0.899521
6,mean_per_class_accuracy,0.752496,0.004632,0.750192,0.750597,0.752817,0.755191,0.753886,0.750259,0.761736,0.743785,0.755185,0.751316
7,mean_per_class_error,0.247504,0.004632,0.249808,0.249403,0.247183,0.244809,0.246114,0.249741,0.238264,0.256215,0.244815,0.248685
8,mse,0.235320,0.002484,0.235282,0.236287,0.232471,0.236022,0.238828,0.232021,0.232277,0.236456,0.234779,0.238778
9,pr_auc,0.973479,0.000782,0.974367,0.973194,0.973752,0.973935,0.972818,0.973262,0.974375,0.971870,0.974080,0.973138


In [16]:
gbm = H2OGradientBoostingEstimator(ntrees = 50, max_depth=20, nfolds=10, seed=1234,auc_type="WEIGHTED_OVO")
gbm.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1717563328635_7


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,650.0,9817886.0,20.0,20.0,19.632307,1.0,1737.0,1196.0431




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.15927990040037418
RMSE: 0.3990988604348228
LogLoss: 0.4368672804770714
Mean Per-Class Error: 0.20188663677009508
AUC: 0.9796053221754447
AUCPR: 0.9804106359899941

Multinomial AUC values: 


,type,first_class_domain,second_class_domain,auc
0,Benign vs Rest,Benign,None,1.000000
1,DNS vs Rest,DNS,None,0.949012
2,LDAP vs Rest,LDAP,None,0.934602
3,MSSQL vs Rest,MSSQL,None,0.930696
4,NTP vs Rest,NTP,None,0.999998
5,NetBIOS vs Rest,NetBIOS,None,0.995577
6,Portmap vs Rest,Portmap,None,0.999942
7,SNMP vs Rest,SNMP,None,0.972765
8,SSDP vs Rest,SSDP,None,0.999804
9,Syn vs Rest,Syn,None,1.000000



See the whole table with table.as_data_frame()

Multinomial auc_pr values: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_classification_error,validation_auc,validation_pr_auc
0,,2024-06-05 06:46:00,43 min 8.887 sec,0.0,0.923077,2.564949,0.940010,0.500000,0.500000,0.923077,2.564949,0.938346,0.500000,0.500000
1,,2024-06-05 06:46:03,43 min 12.555 sec,1.0,0.825636,1.802382,0.240692,0.973453,0.974103,0.825890,1.804832,0.247000,0.971148,0.972081
2,,2024-06-05 06:46:13,43 min 22.445 sec,3.0,0.709259,1.312716,0.236135,0.974417,0.974950,0.710213,1.319039,0.244000,0.972051,0.972770
3,,2024-06-05 06:46:35,43 min 43.831 sec,7.0,0.562390,0.880051,0.217788,0.976654,0.977033,0.565759,0.893789,0.229346,0.973946,0.974442
4,,2024-06-05 06:47:00,44 min 9.626 sec,12.0,0.477853,0.652658,0.214712,0.977863,0.978425,0.483618,0.671774,0.227846,0.975047,0.975649
5,,2024-06-05 06:47:27,44 min 35.971 sec,17.0,0.442305,0.550914,0.211731,0.978380,0.979091,0.449924,0.574398,0.227462,0.975375,0.976106
6,,2024-06-05 06:47:53,45 min 2.079 sec,22.0,0.423314,0.498474,0.207721,0.978933,0.979654,0.432588,0.525930,0.223115,0.975678,0.976377
7,,2024-06-05 06:48:19,45 min 28.038 sec,27.0,0.412911,0.471197,0.205856,0.979197,0.979970,0.423385,0.501827,0.222962,0.975777,0.976484
8,,2024-06-05 06:48:44,45 min 53.467 sec,32.0,0.406681,0.455167,0.203731,0.979364,0.980147,0.417990,0.488511,0.222692,0.975855,0.976514
9,,2024-06-05 06:49:09,46 min 17.920 sec,37.0,0.402722,0.445488,0.202635,0.979525,0.980317,0.414776,0.481181,0.222962,0.975861,0.976499



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Packet Length Min,59695.386719,1.000000,0.221848
1,Fwd Packets Length Total,31480.382812,0.527350,0.116992
2,Init Fwd Win Bytes,30667.669922,0.513736,0.113971
3,Fwd Header Length,29895.271484,0.500797,0.111101
4,Fwd Packet Length Max,19586.341797,0.328105,0.072789
5,Fwd Packet Length Min,17669.701172,0.295998,0.065667
6,Down/Up Ratio,16729.150391,0.280242,0.062171
7,Packet Length Mean,13506.682617,0.226260,0.050195
8,Fwd IAT Total,11155.525391,0.186874,0.041458
9,Flow Packets/s,10690.769531,0.179089,0.039731



See the whole table with table.as_data_frame()


In [17]:
gbm.model_performance(valid).auc()

0.975866324160291

In [18]:
gbmdf=gbm.cross_validation_metrics_summary().as_data_frame()
gbmdf

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
0,accuracy,0.774976,0.004509,0.776355,0.773652,0.776396,0.773638,0.771360,0.777467,0.783368,0.776402,0.775296,0.765828
1,auc,0.975765,0.000714,0.976406,0.975914,0.976241,0.975676,0.974893,0.974959,0.976922,0.975676,0.976188,0.974770
2,err,0.225024,0.004509,0.223645,0.226348,0.223604,0.226362,0.228640,0.222533,0.216632,0.223598,0.224704,0.234172
3,err_count,2340.100000,40.440490,2323.000000,2359.000000,2338.000000,2351.000000,2371.000000,2309.000000,2269.000000,2348.000000,2314.000000,2419.000000
4,logloss,0.481912,0.008847,0.467293,0.479727,0.475342,0.482599,0.487839,0.490080,0.473812,0.478893,0.485790,0.497743
5,max_per_class_error,0.715664,0.029802,0.722846,0.729763,0.751592,0.710199,0.725289,0.643512,0.707031,0.725124,0.699309,0.741974
6,mean_per_class_accuracy,0.775376,0.003193,0.774285,0.770949,0.775152,0.775160,0.775178,0.775154,0.783254,0.775815,0.776231,0.772584
7,mean_per_class_error,0.224624,0.003193,0.225714,0.229051,0.224848,0.224840,0.224822,0.224846,0.216746,0.224185,0.223769,0.227416
8,mse,0.171863,0.002508,0.168698,0.170751,0.170075,0.172096,0.174509,0.172896,0.169435,0.170345,0.173009,0.176813
9,pr_auc,0.976104,0.000579,0.976561,0.975885,0.976375,0.976300,0.975339,0.975336,0.977048,0.976166,0.976548,0.975483


In [21]:
from h2o.estimators import H2OXGBoostEstimator
xgb = H2OXGBoostEstimator(ntrees=50, max_depth=20, nfolds=10, seed=1234,auc_type="WEIGHTED_OVO")
xgb.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid)
# print('without binomial_double_trees:', drf.auc(valid=True))

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1717563328635_9


Model Summary: 


,,number_of_trees
0,,50.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.14090755725908838
RMSE: 0.3753765539549432
LogLoss: 0.38851911158201724
Mean Per-Class Error: 0.17821092235401892
AUC: 0.9836631259100157
AUCPR: 0.9843371962932115

Multinomial AUC values: 


,type,first_class_domain,second_class_domain,auc
0,Benign vs Rest,Benign,None,1.000000
1,DNS vs Rest,DNS,None,0.957507
2,LDAP vs Rest,LDAP,None,0.944090
3,MSSQL vs Rest,MSSQL,None,0.936132
4,NTP vs Rest,NTP,None,0.999998
5,NetBIOS vs Rest,NetBIOS,None,0.998460
6,Portmap vs Rest,Portmap,None,0.999958
7,SNMP vs Rest,SNMP,None,0.979808
8,SSDP vs Rest,SSDP,None,0.999880
9,Syn vs Rest,Syn,None,1.000000



See the whole table with table.as_data_frame()

Multinomial auc_pr values: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_classification_error,validation_auc,validation_pr_auc
0,,2024-06-05 08:15:43,21 min 9.269 sec,0.0,0.923077,2.564949,0.923471,0.500000,0.500000,0.923077,2.564949,0.921500,0.500000,0.500000
1,,2024-06-05 08:15:46,21 min 11.969 sec,1.0,0.721212,1.364389,0.215144,0.978149,0.978810,0.721461,1.367452,0.219038,0.976444,0.977518
2,,2024-06-05 08:15:53,21 min 19.396 sec,4.0,0.526744,0.789354,0.204577,0.980153,0.980780,0.530552,0.802933,0.216000,0.977700,0.978563
3,,2024-06-05 08:16:06,21 min 32.074 sec,9.0,0.425218,0.517671,0.196808,0.981382,0.982059,0.435418,0.544877,0.215269,0.977918,0.978710
4,,2024-06-05 08:16:25,21 min 51.438 sec,16.0,0.392847,0.427104,0.187029,0.982623,0.983259,0.408602,0.465932,0.215769,0.978108,0.978798
5,,2024-06-05 08:16:47,22 min 12.893 sec,24.0,0.382359,0.402705,0.183423,0.983117,0.983773,0.401599,0.449149,0.215308,0.978156,0.978777
6,,2024-06-05 08:17:13,22 min 38.878 sec,34.0,0.378369,0.394425,0.181048,0.983337,0.984003,0.399613,0.445393,0.214654,0.978227,0.978766
7,,2024-06-05 08:17:44,23 min 9.802 sec,46.0,0.375900,0.389547,0.178885,0.983602,0.984274,0.399679,0.446242,0.214769,0.978149,0.978689
8,,2024-06-05 08:17:56,23 min 22.631 sec,50.0,0.375377,0.388519,0.178490,0.983663,0.984337,0.399770,0.446651,0.214769,0.978166,0.978689



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Fwd Packets Length Total,152450.484375,1.000000,0.179897
1,Packet Length Min,140957.859375,0.924614,0.166335
2,Fwd Seg Size Min,84346.929688,0.553274,0.099532
3,Init Fwd Win Bytes,82270.992188,0.539657,0.097083
4,Fwd Packet Length Max,74470.835938,0.488492,0.087878
5,Fwd Header Length,62845.222656,0.412234,0.074160
6,Packet Length Max,46553.824219,0.305370,0.054935
7,Bwd Packets/s,45974.332031,0.301569,0.054251
8,Fwd Packet Length Min,33211.222656,0.217849,0.039190
9,Fwd IAT Total,27550.988281,0.180721,0.032511



See the whole table with table.as_data_frame()


In [22]:
xgb.model_performance(valid).auc()

0.9781662043796268

In [23]:
xgbdf=xgb.cross_validation_metrics_summary().as_data_frame()
drfdf

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
0,accuracy,0.783065,0.004066,0.785212,0.786989,0.784813,0.784614,0.778303,0.781033,0.787665,0.783735,0.783842,0.774443
1,auc,0.978124,0.000537,0.978580,0.978203,0.978339,0.978255,0.977560,0.977901,0.979011,0.978036,0.978297,0.977062
2,err,0.216935,0.004066,0.214788,0.213011,0.215187,0.215386,0.221697,0.218967,0.212335,0.216265,0.216158,0.225557
3,err_count,2256.000000,36.587795,2231.000000,2220.000000,2250.000000,2237.000000,2299.000000,2272.000000,2224.000000,2271.000000,2226.000000,2330.000000
4,logloss,0.451703,0.008216,0.439031,0.447614,0.445965,0.450071,0.460659,0.455199,0.445658,0.453755,0.451211,0.467866
5,max_per_class_error,0.621548,0.017589,0.642401,0.612903,0.605583,0.603495,0.629893,0.630566,0.590331,0.631646,0.625984,0.642683
6,mean_per_class_accuracy,0.783297,0.002456,0.782438,0.783555,0.783427,0.786099,0.783120,0.779442,0.788190,0.783206,0.782732,0.780758
7,mean_per_class_error,0.216703,0.002456,0.217562,0.216445,0.216573,0.213901,0.216880,0.220558,0.211810,0.216794,0.217268,0.219242
8,mse,0.161427,0.002768,0.158629,0.159275,0.160161,0.162003,0.164743,0.162022,0.158567,0.161522,0.160147,0.167204
9,pr_auc,0.978575,0.000438,0.978907,0.978516,0.978737,0.978753,0.977977,0.978464,0.979349,0.978455,0.978745,0.977841


In [11]:
type(drf.scoring_history())

AttributeError: 'H2OAutoML' object has no attribute 'scoring_history'

In [ ]:
pd.DataFrame(drf.as_data_frame())

In [ ]:
pd.DataFrame(drf)
#as.data.frame(table(drf))

In [ ]:
pd.DataFrame(drf.varimp())

In [ ]:
from h2o.automl import H2OAutoML
#aml = H2OAutoML(max_models = 10, max_runtime_secs=100, seed = 1)
aml = H2OAutoML(max_models = 10, seed = 1)
aml.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

In [10]:
lb = aml.leaderboard
lb

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_1_AutoML_1_20240605_51316,0.218379,0.44887,0.40154,0.161234


In [ ]:
import pandas as pd
df_train=pd.read_csv("/kaggle/input/cicddos2019only11catlabelbalanced-train/CICDDoS2019Only12CatLabelBalanced_train.csv")
#df_test=pd.read_csv("/kaggle/input/cicddos2019reducedcatlabelbalanced-train/CICDDoS2019CatLabel_test7Atk.csv")
print(df_train.Label.value_counts())
#df_test.Label.value_counts()